In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from numpy.random import normal
import math
import usefulFunctions
from usefulFunctions import checkForNull
from IPython.core.debugger import Tracer
import random
import numpy
from statsmodels.formula.api import ols
from usefulFunctions import computeMean
from usefulFunctions import computeStdev

In [2]:
fileName = "Master_PL.xlsx - Master_PL.csv"
table = pd.read_csv(fileName, header=0, 
                    names=["id", "type", "gender", "age", "monthsFromScreening", "protein_BL", "protein_V4",
                          "protein_V6", "protein_V8", "protein_V10", "protein_V12"])


In [141]:
dataWithType = {}
for entry in ["id", "type", "gender", "age", "monthsFromEval", "protein"]:
    dataWithType[entry] = []

dataWithMonthsFromScreening = {}
for entry in ["id", "gender", "age", "monthsFromScreening", "monthsFromEval", "protein"]:
    dataWithMonthsFromScreening[entry] = []

for row in table.to_dict(orient='records'):

    for entry in ["protein_BL", "protein_V4","protein_V6", "protein_V8", "protein_V10", "protein_V12"]:
        if not checkForNull(row[entry]):
            cont1 = True
            cont2 = True
            
            for key in ["id", "type", "gender", "age"]:
                if checkForNull(row[key]):
                    cont1 = False
            
            for key in ["id", "gender", "age", "monthsFromScreening"]:
                if checkForNull(row[key]):
                    cont2 = False
            
            if cont1:
                if entry == "protein_BL":
                    dataWithType["monthsFromEval"].append(0.0)
                elif entry == "protein_V4":
                    dataWithType["monthsFromEval"].append(4.0)
                elif entry == "protein_V6":
                    dataWithType["monthsFromEval"].append(6.0)
                elif entry == "protein_V8":
                    dataWithType["monthsFromEval"].append(8.0)
                elif entry == "protein_V10":
                    dataWithType["monthsFromEval"].append(10.0)
                elif entry == "protein_V12":
                    dataWithType["monthsFromEval"].append(12.0)

                dataWithType['protein'].append(float(row[entry]))
                
                for key in ["id", "type", "gender", "age"]:
                    if key == "type" or key == "gender":
                        dataWithType[key].append(row[key]-1)
                    else:
                        dataWithType[key].append(row[key])
            
            if cont2:

                if entry == "protein_BL":
                    dataWithMonthsFromScreening["monthsFromEval"].append(0.0)
                elif entry == "protein_V4":
                    dataWithMonthsFromScreening["monthsFromEval"].append(4.0)
                elif entry == "protein_V6":
                    dataWithMonthsFromScreening["monthsFromEval"].append(6.0)
                elif entry == "protein_V8":
                    dataWithMonthsFromScreening["monthsFromEval"].append(8.0)
                elif entry == "protein_V10":
                    dataWithMonthsFromScreening["monthsFromEval"].append(10.0)
                elif entry == "protein_V12":
                    dataWithMonthsFromScreening["monthsFromEval"].append(12.0)

                dataWithMonthsFromScreening['protein'].append(float(row[entry]))

                for key in ["id", "gender", "age", "monthsFromScreening"]:
                    if key == "gender":
                        dataWithMonthsFromScreening[key].append(row[key]-1)
                    else:
                        dataWithMonthsFromScreening[key].append(row[key])

In [42]:
dataWithType = pd.DataFrame(dataWithType)
dataWithMonthsFromScreening = pd.DataFrame(dataWithMonthsFromScreening)

In [134]:
len(dataWithType)

2162

In [135]:
len(dataWithMonthsFromScreening)

1378

In [137]:
statsForDataWithType = {}
for column in dataWithType:
    statsForDataWithType[column] = {"mean" : computeMean(dataWithType[column]), "stdev" : computeStdev(dataWithType[column])}
    
statsForDataWithMonthsFromScreening = {}
for column in dataWithMonthsFromScreening:
     statsForDataWithMonthsFromScreening[column] = {"mean" : computeMean(dataWithMonthsFromScreening[column]), "stdev" : computeStdev(dataWithMonthsFromScreening[column])}


In [142]:
# CODE TO STANDARDIZE INDEPENDENT VARIABLES
# for i in range(len(dataWithType['age'])):
#     dataWithType['age'][i] = (dataWithType['age'][i]-statsForDataWithType['age']['mean'])/statsForDataWithType['age']['stdev']
#     dataWithType['monthsFromEval'][i] = (dataWithType['monthsFromEval'][i]-statsForDataWithType['monthsFromEval']['mean'])/statsForDataWithType['monthsFromEval']['stdev']
    
# for i in range(len(dataWithMonthsFromScreening['age'])):
#     dataWithMonthsFromScreening['age'][i] = (dataWithMonthsFromScreening['age'][i]-statsForDataWithMonthsFromScreening['age']['mean'])/statsForDataWithMonthsFromScreening['age']['stdev']
#     dataWithMonthsFromScreening['monthsFromScreening'][i] = (dataWithMonthsFromScreening['monthsFromScreening'][i]-statsForDataWithMonthsFromScreening['monthsFromScreening']['mean'])/statsForDataWithMonthsFromScreening['monthsFromScreening']['stdev']
#     dataWithMonthsFromScreening['monthsFromEval'][i] = (dataWithMonthsFromScreening['monthsFromEval'][i]-statsForDataWithMonthsFromScreening['monthsFromEval']['mean'])/statsForDataWithMonthsFromScreening['monthsFromEval']['stdev']
     
    

In [145]:
modelWithType = ols('protein ~ type + gender + age + monthsFromEval', dataWithType).fit()
modelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age + monthsFromEval', dataWithMonthsFromScreening).fit()

&&&&

Independent : Type (Case vs Control), Gender, Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [44]:
print(modelWithType.summary()) 

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     43.71
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           2.54e-35
Time:                        00:49:13   Log-Likelihood:                -9841.2
No. Observations:                2162   AIC:                         1.969e+04
Df Residuals:                    2157   BIC:                         1.972e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         20.8955      3.210      6.

&&&&

Independent : PD Duration (monthsFromScreening), Gender, Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [46]:
print(modelWithMonthsFromScreening.summary()) 

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     27.34
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           7.25e-22
Time:                        00:49:17   Log-Likelihood:                -6348.9
No. Observations:                1378   AIC:                         1.271e+04
Df Residuals:                    1373   BIC:                         1.273e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              22.0772    

In [47]:
malesDataWithType = dataWithType[dataWithType['gender'] == 1]
femalesDataWithType = dataWithType[dataWithType['gender'] == 0]

In [48]:
maleModelWithType = ols('protein ~ type + age + monthsFromEval', malesDataWithType).fit()
femaleModelWithType = ols('protein ~ type + age + monthsFromEval', femalesDataWithType).fit()

&&&&

MALE DATA ONLY

Independent : Type (Case vs Control), Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [140]:
print(maleModelWithType.summary()) 

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     7.397
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           6.44e-05
Time:                        11:59:38   Log-Likelihood:                -6648.7
No. Observations:                1430   AIC:                         1.331e+04
Df Residuals:                    1426   BIC:                         1.333e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         29.5726      4.429      6.

&&&&

FEMALE DATA ONLY

Independent : Type (Case vs Control), Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [50]:
print(femaleModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.494
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0589
Time:                        00:49:31   Log-Likelihood:                -3129.6
No. Observations:                 732   AIC:                             6267.
Df Residuals:                     728   BIC:                             6286.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         27.1472      3.975      6.

In [51]:
malesDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['gender'] == 1]
femalesDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['gender'] == 0]

In [52]:
maleModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + age + monthsFromEval', malesDataWithMonthsFromScreening).fit()
femaleModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + age + monthsFromEval', femalesDataWithMonthsFromScreening).fit()

&&&&

MALE DATA ONLY

Independent : PD Duration(monthsFromScreening), Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [53]:
print(maleModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     4.979
Date:                Thu, 27 Jul 2017   Prob (F-statistic):            0.00197
Time:                        00:49:45   Log-Likelihood:                -4295.2
No. Observations:                 918   AIC:                             8598.
Df Residuals:                     914   BIC:                             8618.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              28.1929    

&&&&

FEMALE DATA ONLY

Independent : PD Duration(monthsFromScreening), Age, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [54]:
print(femaleModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.2328
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.873
Time:                        00:49:52   Log-Likelihood:                -2031.7
No. Observations:                 460   AIC:                             4071.
Df Residuals:                     456   BIC:                             4088.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              33.8179    

In [64]:
youngWithType = dataWithType[dataWithType['age'] < 50]
midOneWithType = dataWithType[dataWithType['age'] < 60][dataWithType['age'] >= 50]
midTwoWithType = dataWithType[dataWithType['age'] < 65][dataWithType['age'] >= 60]
midThreeWithType = dataWithType[dataWithType['age'] < 70][dataWithType['age'] >= 65]
oldWithType = dataWithType[dataWithType['age'] >= 70]
print len(youngWithType), len(midOneWithType), len(midTwoWithType), len(midThreeWithType), len(oldWithType)

youngModelWithType = ols('protein ~ type + gender + monthsFromEval', youngWithType).fit()
midOneModelWithType = ols('protein ~ type + gender + monthsFromEval', midOneWithType).fit()
midTwoModelWithType = ols('protein ~ type + gender + monthsFromEval', midTwoWithType).fit()
midThreeModelWithType = ols('protein ~ type + gender + monthsFromEval', midThreeWithType).fit()
oldModelWithType = ols('protein ~ type + gender + monthsFromEval', oldWithType).fit()

268 617 397 427 453


/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


&&&&

YOUNG DATA ONLY (age < 50)

Independent : Type (Case Vs Control), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [65]:
print(youngModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.061
Method:                 Least Squares   F-statistic:                     6.825
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           0.000191
Time:                        00:55:34   Log-Likelihood:                -1133.0
No. Observations:                 268   AIC:                             2274.
Df Residuals:                     264   BIC:                             2288.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         31.1528      2.532     12.

&&&&

MID_1 DATA ONLY (50 <= age < 60)

Independent : Type (Case Vs Control), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [66]:
print(midOneModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     19.29
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           5.82e-12
Time:                        00:55:38   Log-Likelihood:                -2706.9
No. Observations:                 617   AIC:                             5422.
Df Residuals:                     613   BIC:                             5440.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         36.1705      2.025     17.

&&&&

MID_2 DATA ONLY (60 <= age < 65)

Independent : Type (Case Vs Control), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [67]:
print(midTwoModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     17.32
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           1.40e-10
Time:                        00:55:58   Log-Likelihood:                -1729.6
No. Observations:                 397   AIC:                             3467.
Df Residuals:                     393   BIC:                             3483.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         32.9213      2.426     13.

&&&&

MID_3 DATA ONLY (65 <= age < 70)

Independent : Type (Case Vs Control), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [68]:
print(midThreeModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.065
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     9.829
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           2.80e-06
Time:                        01:00:59   Log-Likelihood:                -2070.4
No. Observations:                 427   AIC:                             4149.
Df Residuals:                     423   BIC:                             4165.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         29.6058      4.308      6.

&&&&

OLD DATA ONLY (age >= 70)

Independent : Type (Case Vs Control), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [69]:
print(oldModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     7.212
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           9.78e-05
Time:                        01:01:07   Log-Likelihood:                -2088.3
No. Observations:                 453   AIC:                             4185.
Df Residuals:                     449   BIC:                             4201.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         35.1310      3.052     11.

In [78]:
youngWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['age'] < 50]
midOneWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['age'] < 60][dataWithMonthsFromScreening['age'] >= 50]
midTwoWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['age'] < 65][dataWithMonthsFromScreening['age'] >= 60]
midThreeWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['age'] < 70][dataWithMonthsFromScreening['age'] >= 65]
oldWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithType['age'] >= 70]
print len(youngWithMonthsFromScreening), len(midOneWithMonthsFromScreening), len(midTwoWithMonthsFromScreening), len(midThreeWithMonthsFromScreening), len(oldWithMonthsFromScreening)

youngModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + monthsFromEval', youngWithMonthsFromScreening).fit()
midOneModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + monthsFromEval', midOneWithMonthsFromScreening).fit()
midTwoModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + monthsFromEval', midTwoWithMonthsFromScreening).fit()
midThreeModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + monthsFromEval', midThreeWithMonthsFromScreening).fit()
oldModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + monthsFromEval', oldWithMonthsFromScreening).fit()

166 384 241 306 199


/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/Users/rahulkhanna/Documents/comp_gene/clouds/lib/python2.7/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


&&&&

YOUNG DATA ONLY (age < 50)

Independent : PD Duration (monthsFromScreening), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [79]:
print(youngModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     2.731
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0457
Time:                        10:39:14   Log-Likelihood:                -715.37
No. Observations:                 166   AIC:                             1439.
Df Residuals:                     162   BIC:                             1451.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              34.3665    

&&&&

MID_1 DATA ONLY (50 <= age < 60)

Independent : PD Duration (monthsFromScreening), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [80]:
print(midOneModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     6.325
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           0.000340
Time:                        10:39:19   Log-Likelihood:                -1741.1
No. Observations:                 384   AIC:                             3490.
Df Residuals:                     380   BIC:                             3506.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              34.8665    

&&&&

MID_2 DATA ONLY (60 <= age < 65)

Independent : PD Duration (monthsFromScreening), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [81]:
print(midTwoModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     12.84
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           8.40e-08
Time:                        10:39:23   Log-Likelihood:                -1070.7
No. Observations:                 241   AIC:                             2149.
Df Residuals:                     237   BIC:                             2163.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              35.8813    

&&&&

MID_3 DATA ONLY (65 <= age < 70)

Independent : PD Duration (monthsFromScreening), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [82]:
print(midThreeModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     15.63
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           1.79e-09
Time:                        10:39:28   Log-Likelihood:                -1431.0
No. Observations:                 306   AIC:                             2870.
Df Residuals:                     302   BIC:                             2885.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              29.3312    

&&&&

OLD DATA ONLY (age >= 70)

Independent : PD Duration (monthsFromScreening), Gender, MonthsFromEval (BL, 4 months, 6 months, etc)

Dependent : Protein Level

In [83]:
print(oldModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.089
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     6.360
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           0.000391
Time:                        10:39:33   Log-Likelihood:                -965.03
No. Observations:                 199   AIC:                             1938.
Df Residuals:                     195   BIC:                             1951.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              33.6835    

In [85]:
blDataWithType = dataWithType[dataWithType['monthsFromEval'] == 0]
fourDataWithType = dataWithType[dataWithType['monthsFromEval'] == 4]
sixDataWithType = dataWithType[dataWithType['monthsFromEval'] == 6]
eightDataWithType = dataWithType[dataWithType['monthsFromEval'] == 8]
tenDataWithType = dataWithType[dataWithType['monthsFromEval'] == 10]
twelveDataWithType = dataWithType[dataWithType['monthsFromEval'] == 12]

blModelWithType = ols('protein ~ type + gender + age', blDataWithType).fit()
fourModelWithType = ols('protein ~ type + gender + age', fourDataWithType).fit()
sixModelWithType = ols('protein ~ type + gender + age', sixDataWithType).fit()
eightModelWithType = ols('protein ~ type + gender + age', eightDataWithType).fit()
tenModelWithType = ols('protein ~ type + gender + age', tenDataWithType).fit()
twelveModelWithType = ols('protein ~ type + gender + age', twelveDataWithType).fit()

&&&&

ONLY PROTEIN LEVELS FROM THE BL EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [86]:
print(blModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.099
Method:                 Least Squares   F-statistic:                     21.92
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           1.92e-13
Time:                        10:53:53   Log-Likelihood:                -2475.1
No. Observations:                 571   AIC:                             4958.
Df Residuals:                     567   BIC:                             4976.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     24.6002      4.817      5.107      0.0

&&&&

ONLY PROTEIN LEVELS FROM THE 4 MONTH EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [87]:
print(fourModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     11.28
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           3.91e-07
Time:                        10:54:39   Log-Likelihood:                -2000.5
No. Observations:                 430   AIC:                             4009.
Df Residuals:                     426   BIC:                             4025.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.6805      7.830      2.514      0.0

&&&&

ONLY PROTEIN LEVELS FROM THE SIX MONTH EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [88]:
print(sixModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     4.280
Date:                Thu, 27 Jul 2017   Prob (F-statistic):            0.00545
Time:                        10:55:00   Log-Likelihood:                -1947.0
No. Observations:                 402   AIC:                             3902.
Df Residuals:                     398   BIC:                             3918.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     27.6946      9.802      2.825      0.0

&&&&

ONLY PROTEIN LEVELS FROM THE EIGHT MONTH EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [89]:
print(eightModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.099
Model:                            OLS   Adj. R-squared:                  0.092
Method:                 Least Squares   F-statistic:                     13.91
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           1.26e-08
Time:                        10:55:19   Log-Likelihood:                -1676.5
No. Observations:                 384   AIC:                             3361.
Df Residuals:                     380   BIC:                             3377.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.7344      6.417      3.075      0.0

&&&&

ONLY PROTEIN LEVELS FROM THE TEN MONTH EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [90]:
print(tenModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.134
Method:                 Least Squares   F-statistic:                     13.98
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           1.87e-08
Time:                        10:55:36   Log-Likelihood:                -1083.7
No. Observations:                 253   AIC:                             2175.
Df Residuals:                     249   BIC:                             2189.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     17.8157      7.012      2.541      0.0

&&&&

ONLY PROTEIN LEVELS FROM THE TWELVE MONTH EVAL

Independent : Type (Case vs Control), Gender, Age

Dependent : Protein Level

In [91]:
print(twelveModelWithType.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     3.549
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0167
Time:                        10:55:53   Log-Likelihood:                -558.08
No. Observations:                 122   AIC:                             1124.
Df Residuals:                     118   BIC:                             1135.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     22.0754     13.031      1.694      0.0

In [111]:
blDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 0]
fourDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 4]
sixDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 6]
eightDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 8]
tenDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 10]
twelveDataWithMonthsFromScreening = dataWithMonthsFromScreening[dataWithMonthsFromScreening['monthsFromEval'] == 12]

blModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', blDataWithMonthsFromScreening).fit()
fourModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', fourDataWithMonthsFromScreening).fit()
sixModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', sixDataWithMonthsFromScreening).fit()
eightModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', eightDataWithMonthsFromScreening).fit()
tenModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', tenDataWithMonthsFromScreening).fit()
twelveModelWithMonthsFromScreening = ols('protein ~ monthsFromScreening + gender + age', twelveDataWithMonthsFromScreening).fit()

&&&&

ONLY PROTEIN LEVELS FROM THE BL EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [112]:
print(blModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     10.92
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           7.00e-07
Time:                        11:45:29   Log-Likelihood:                -1617.2
No. Observations:                 365   AIC:                             3242.
Df Residuals:                     361   BIC:                             3258.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              26.3502    

&&&&

ONLY PROTEIN LEVELS FROM THE FOUR MONTH EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [113]:
print(fourModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     6.462
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           0.000307
Time:                        11:45:45   Log-Likelihood:                -1309.5
No. Observations:                 274   AIC:                             2627.
Df Residuals:                     270   BIC:                             2641.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              23.1580    

&&&&

ONLY PROTEIN LEVELS FROM THE SIX MONTH EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [114]:
print(sixModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     2.417
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0668
Time:                        11:45:45   Log-Likelihood:                -1225.7
No. Observations:                 258   AIC:                             2459.
Df Residuals:                     254   BIC:                             2474.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              33.0993    

&&&&

ONLY PROTEIN LEVELS FROM THE EIGHT MONTH EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [115]:
print(eightModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.103
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     9.575
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           5.25e-06
Time:                        11:45:45   Log-Likelihood:                -1131.0
No. Observations:                 253   AIC:                             2270.
Df Residuals:                     249   BIC:                             2284.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              22.6923    

&&&&

ONLY PROTEIN LEVELS FROM THE TEN MONTH EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [116]:
print(tenModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     9.817
Date:                Thu, 27 Jul 2017   Prob (F-statistic):           5.53e-06
Time:                        11:45:45   Log-Likelihood:                -721.63
No. Observations:                 165   AIC:                             1451.
Df Residuals:                     161   BIC:                             1464.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              13.0677    

&&&&

ONLY PROTEIN LEVELS FROM THE TWELVE MONTH EVAL

Independent : PD DURATION (monthsFromScreening), Gender, Age

Dependent : Protein Level

In [117]:
print(twelveModelWithMonthsFromScreening.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     2.097
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.110
Time:                        11:45:45   Log-Likelihood:                -300.21
No. Observations:                  63   AIC:                             608.4
Df Residuals:                      59   BIC:                             617.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              14.7275    

In [118]:
maleBLDataWithType = blDataWithType[blDataWithType['gender']==1]
femaleBLDataWithType = blDataWithType[blDataWithType['gender']==0]
maleFourDataWithType = fourDataWithType[fourDataWithType['gender']==1]
femaleFourDataWithType = fourDataWithType[fourDataWithType['gender']==0]
maleSixDataWithType = sixDataWithType[sixDataWithType['gender']==1]
femaleSixDataWithType = sixDataWithType[sixDataWithType['gender']==0]
maleEightDataWithType = eightDataWithType[eightDataWithType['gender']==1]
femaleEightDataWithType = eightDataWithType[eightDataWithType['gender']==0]
maleTenDataWithType = tenDataWithType[tenDataWithType['gender']==1]
femaleTenDataWithType = tenDataWithType[tenDataWithType['gender']==0]
maleTwelveDataWithType = twelveDataWithType[twelveDataWithType['gender']==1]
femaleTwelveDataWithType = twelveDataWithType[twelveDataWithType['gender']==0]
print len(maleBLDataWithType), len(maleFourDataWithType), len(maleEightDataWithType), len(maleTenDataWithType), len(maleTwelveDataWithType)
print len(femaleBLDataWithType), len(femaleFourDataWithType), len(femaleEightDataWithType), len(femaleTenDataWithType), len(femaleTwelveDataWithType)

maleBLModel = ols('protein ~ type + age', maleBLDataWithType).fit()
maleFourModel = ols('protein ~ type + age', maleFourDataWithType).fit()
maleSixModel = ols('protein ~ type + age', maleSixDataWithType).fit()
maleEightModel = ols('protein ~ type + age', maleEightDataWithType).fit()
maleTenModel = ols('protein ~ type + age', maleTenDataWithType).fit()
maleTwelveModel = ols('protein ~ type + age', maleTwelveDataWithType).fit()

femaleBLModel = ols('protein ~ type + age', femaleBLDataWithType).fit()
femaleFourModel = ols('protein ~ type + age', femaleFourDataWithType).fit()
femaleSixModel = ols('protein ~ type + age', femaleSixDataWithType).fit()
femaleEightModel = ols('protein ~ type + age', femaleEightDataWithType).fit()
femaleTenModel = ols('protein ~ type + age', femaleTenDataWithType).fit()
femaleTwelveModel = ols('protein ~ type + age', femaleTwelveDataWithType).fit()

374 289 253 171 78
197 141 131 82 44


&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE BL EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [119]:
print(maleBLModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.146
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.118
Time:                        11:45:45   Log-Likelihood:                -1668.9
No. Observations:                 374   AIC:                             3344.
Df Residuals:                     371   BIC:                             3356.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.4612      6.942      4.820      0.0

&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE 4 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [120]:
print(maleFourModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     2.306
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.101
Time:                        11:45:45   Log-Likelihood:                -1388.3
No. Observations:                 289   AIC:                             2783.
Df Residuals:                     286   BIC:                             2794.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     26.6167     11.410      2.333      0.0

&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE 6 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [121]:
print(maleSixModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.035
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.357
Time:                        11:45:45   Log-Likelihood:                -1285.1
No. Observations:                 265   AIC:                             2576.
Df Residuals:                     262   BIC:                             2587.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.5862     12.198      2.671      0.0

&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE 8 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [122]:
print(maleEightModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     4.132
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0172
Time:                        11:45:45   Log-Likelihood:                -1132.0
No. Observations:                 253   AIC:                             2270.
Df Residuals:                     250   BIC:                             2281.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     29.0697      8.858      3.282      0.0

&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE 10 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [123]:
print(maleTenModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     2.898
Date:                Thu, 27 Jul 2017   Prob (F-statistic):             0.0579
Time:                        11:45:45   Log-Likelihood:                -754.89
No. Observations:                 171   AIC:                             1516.
Df Residuals:                     168   BIC:                             1525.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     32.0702     10.242      3.131      0.0

&&&&

MALE DATA

ONLY PROTEIN LEVELS FROM THE 12 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [124]:
print(maleTwelveModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.7664
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.468
Time:                        11:45:45   Log-Likelihood:                -369.05
No. Observations:                  78   AIC:                             744.1
Df Residuals:                      75   BIC:                             751.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     34.5655     18.243      1.895      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE BL EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [132]:
print(femaleBLModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9969
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.371
Time:                        11:49:22   Log-Likelihood:                -773.54
No. Observations:                 197   AIC:                             1553.
Df Residuals:                     194   BIC:                             1563.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     30.2948      5.135      5.899      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE 4 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [133]:
print(femaleFourModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.2780
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.758
Time:                        11:49:22   Log-Likelihood:                -561.94
No. Observations:                 141   AIC:                             1130.
Df Residuals:                     138   BIC:                             1139.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     31.4918      6.627      4.752      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE 6 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [127]:
print(femaleSixModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.3138
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.731
Time:                        11:45:46   Log-Likelihood:                -661.09
No. Observations:                 137   AIC:                             1328.
Df Residuals:                     134   BIC:                             1337.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     37.2630     15.972      2.333      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE 8 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [128]:
print(femaleEightModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.652
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.196
Time:                        11:45:46   Log-Likelihood:                -527.20
No. Observations:                 131   AIC:                             1060.
Df Residuals:                     128   BIC:                             1069.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     25.1452      7.537      3.336      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE 10 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [129]:
print(femaleTenModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     6.073
Date:                Thu, 27 Jul 2017   Prob (F-statistic):            0.00352
Time:                        11:45:46   Log-Likelihood:                -307.85
No. Observations:                  82   AIC:                             621.7
Df Residuals:                      79   BIC:                             628.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.3202      6.628      2.311      0.0

&&&&

FEMALE DATA

ONLY PROTEIN LEVELS FROM THE 12 MONTH EVAL

Independent : Type (Case vs Control), Age

Dependent : Protein Level

In [143]:
print(femaleTwelveModel.summary())

                            OLS Regression Results                            
Dep. Variable:                protein   R-squared:                       0.067
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.475
Date:                Thu, 27 Jul 2017   Prob (F-statistic):              0.241
Time:                        12:24:36   Log-Likelihood:                -176.58
No. Observations:                  44   AIC:                             359.2
Df Residuals:                      41   BIC:                             364.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     19.9515     13.440      1.485      0.1